In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./2.url.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests

        import sys
        sys.path.append('../00.Tools')
        from crawler_configuration import get_header, get_proxy

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.get(input_.loc[a, 'Url'],
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree

            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            title = html.xpath('//head/title/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            subtitle = html.xpath('//h2[@itemprop="brand"]/a/span[@itemprop="name"]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            part_number = html.xpath('//dd[@itemprop="sku"]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            part_type = html.xpath('//dd[@data-product-parttype]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            dict_src = {}
            list_src = html.xpath('//li[contains(@class, "productView-thumbnail")]/a[@class="productView-thumbnail-link"]/@href')
            for i, src in zip(range(len(list_src)), list_src):
                dict_src[str(i)] = src.strip()

            # = = = = = = = = = = = = = = =

            import json

            dict_info = {}
            list_dict = json.loads(html.xpath('//div[@id="addon"]/@data-custom-fields')[0])
            for i, dict_ in zip(range(len(list_dict)), list_dict):
                dict_info[str(i)] = {dict_['name'].strip(): dict_['value'].strip()}

            # = = = = = = = = = = = = = = =

            bundle = resp.text.split('window.__webpack_public_path__ = "')[1].split('";</script>')[0]

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Part Type': part_type,
                                     'Part Number': part_number,
                                     'Title': title,
                                     'Subtitle': subtitle,
                                     'Json_Info': json.dumps(dict_info),
                                     'Url': input_.loc[a, 'Url'],
                                     'Json_Src': json.dumps(dict_src),
                                     'Bundle': bundle}])

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Page': input_.loc[a, 'Page'],
                                     'Page Url': input_.loc[a, 'Page Url'],
                                     'No': input_.loc[a, 'No'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./3.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：2] - 394.https://www.cardone.com/product/windshield-wiper-motor-43-1153
[剩余数量：1000] - [当前时间：17:45:36]

[状态：ok，尝试次数：2] - 399.https://www.cardone.com/product/windshield-wiper-motor-43-1163
[剩余数量：999] - [当前时间：17:45:37]

[状态：ok，尝试次数：4] - 396.https://www.cardone.com/product/windshield-wiper-motor-43-1159
[剩余数量：998] - [当前时间：17:45:37]

[状态：ok，尝试次数：3] - 400.https://www.cardone.com/product/windshield-wiper-motor-43-1164
[剩余数量：997] - [当前时间：17:45:37]

[状态：ok，尝试次数：1] - 401.https://www.cardone.com/product/windshield-wiper-motor-43-1165
[剩余数量：996] - [当前时间：17:45:38]

[状态：ok，尝试次数：5] - 397.https://www.cardone.com/product/windshield-wiper-motor-43-1160
[剩余数量：995] - [当前时间：17:45:38]

[状态：ok，尝试次数：3] - 402.https://www.cardone.com/product/windshield-wiper-motor-43-1166
[剩余数量：994] - [当前时间：17:45:39]

[状态：ok，尝试次数：2] - 404.https://www.cardone.com/product/windshield-wiper-motor-43-1169
[剩余数量：993] - [当前时间：17:45:39]

[状态：ok，尝试次数：20] - 398.https://www.cardone.com/product/windshield-wiper-motor-43-1162
[剩